# Practical: String basics

In [ ]:
# Concatenation
seqs = ['A', 'C', 'G', 'T']
print ''.join(seqs) #ACGT

In [ ]:
# Random string
import random
#random.seed(7) # same every time
random.choice('ACGT')

seq = ''
# underscore means you don't care about storing an index
for _ in range(10):
    seq += random.choice('ACGT')
print(seq)

# anoter way of generation random
seq = ''.join([random.choice('ACGT') for _ in range (10)])
print seq

In [ ]:
# range of characters
print seq[1:3] # starting at 1, not including 3
print seq[:3] # up to 3
print seq[7:] # 7 up to end
print seq[-3] # 3rd to last
print seq[-3:] # from 3rd to last until end


# Practical: Manipulating DNA strings

In [ ]:
# Function to find the longest common prefix between two strings

def longestCommonPrefix(s1, s2):
    i = 0
    while i < len(s1) and i < len(s2) and s1[i] == s2[i]:
        i += 1
    return s1[:i]

longestCommonPrefix('ACCATGT', 'ACCAGAC')

In [ ]:
# Function to see if two strings match exactly
def match(s1, s2):
    if not len(s1) == len(s2):
        return False
    for i in range(len(s1)):
        if not s1[i] == s2[i]:
            return False
    return True

print match('ATGCT', 'ATGCT')
print match('ATCGT', 'ATCGA')

# but also...
print 'ATCGT' == 'ATCGT'
print 'ATCGT' == 'ATCGA'

In [ ]:
# Function to find reverse compliment


def reverseComplement(s):
    complement = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
    t = ''
    for base in s:
        t = complement[base] + t # add to beginning, to reverse the string
    return t

reverseComplement('ACCGTCG')

# Practical: Downloading and parsing a genome

In [ ]:
# getting file!
# exclamation point lets you run command line in notebook
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa

In [ ]:
# Function to read genome from fasta
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('lambda_virus.fa')


In [ ]:
# Counting base pairs

# Method 1
counts = {'A': 0, 'C':0, 'G':0, 'T':0}
for base in genome:
    counts[base] += 1
    
# Method 2
import collections
collections.Counter(genome)

# Practical: Working with sequencing reads

In [ ]:
# Get FastQ file
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/SRR835775_1.first1000.fastq

In [ ]:
# Function to read/parse FastQ file

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break # reached the end
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

seqs, quals = readFastq('SRR835775_1.first1000.fastq')


In [ ]:
# Create histogram of quality scores

# Convert Q score to Phred33 ASCII-econded quality
def QtoPhred33(Q):
    return chr(Q + 33)

# Convert Phred33 to Q
def phred33ToQ(qual):
    return ord(qual) - 33

# Histogram creator
def createHist(qualities):
    hist = [0] * 50
    for qual in qualities:
        for phred in qual:
            q = phred33ToQ(phred)
            hist[q] += 1
    return hist

h = createHist(quals)
print h

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.bar(range(len(h)), h)
plt.show()

# Practical: Analyzing reads by position

In [ ]:
# Plot CG content at each position - varies by species
# Mix shouldn't change along the read, check for weirdness

def findGCByPos(reads):
    gc = [0] * 100 # reads are length 100
    totals = [0] * 100
    for read in reads:
        for i in range(len(read)):
            if read[i] == 'C' or read[i] == 'G':
                gc[i] += 1
            totals[i] += 1
    for i in range(len(gc)):
        if totals[i] > 0:
            gc[i] /= float(totals[i])
            
    return gc

gc = findGCByPos(seqs)
plt.plot(range(len(gc)), gc)
plt.show()

In [ ]:
# Look at distribution of bases 
import collections
count = collections.Counter()
for seq in seqs:
    count.update(seq)
print count
# N is when the base caller has no confidence

# Practical: Matching artificial reads

In [ ]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa


In [ ]:
# from above
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('phix.fa')

# Naive matching
def naive(p, t):
    occurrences = []
    # Loop through alignments
    for i in range(len(t) - len(p) + 1):
        match = True
        # Loop through characters
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
          occurrences.append(i)
    return occurrences

t = 'AGCTTAGATAGC'
p = 'AG'
naive(p, t)

In [ ]:
import random
# generate artifical reads from a genome
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

# Generate 100 reads of length 100
reads = generateReads(genome, 100, 100)

# find proportion of matches
numMatched = 0
for r in reads:
    matches = naive(r, genome)
    if len(matches) > 0:
        numMatched += 1
        
print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))        

# Practical: matching real reads

In [ ]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
# get reads

In [18]:
# From before
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break # reached the end
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

phix_reads, _ = readFastq('ERR266411_1.first1000.fastq')

In [19]:
# Same as above but with real
numMatched = 0
n = 0
for r in phix_reads:
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n))
# only 7/1000 actually match...
# could be error (mostly) but could also be variability 

7 / 1000 reads matched the genome!


In [20]:
# try with 1st 30 bases
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30] 
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n))
# not looking at reverse compliment

459 / 1000 reads matched the genome!


In [22]:
# include reverse compliment
def reverseComplement(s):
    complement = {'A':'T', 'T':'A', 'G':'C', 'C':'G', 'N':'N'}
    t = ''
    for base in s:
        t = complement[base] + t # add to beginning, to reverse the string
    return t

numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30] 
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome!' % (numMatched, n))

932 / 1000 reads matched the genome!
